In [7]:
import pdfplumber
import csv

def extract_tables_from_pdf(pdf_path, output_csv_path):
    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []
        # Iterate through each page of the PDF
        for page in pdf.pages:
            # Extract tables from the current page
            tables = page.extract_tables()
            for table in tables:
                all_tables.extend(table)  # Add the rows of the table to all_tables list
       
        # Write all extracted tables to a CSV file
        with open(output_csv_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for row in all_tables:
                writer.writerow(row)

# Specify the path to your PDF and the output CSV file
pdf_path = 'Data/2020-Table-of-Fees-25Jun20.pdf'
output_csv_path = 'Data/2020_Fees.csv'

extract_tables_from_pdf(pdf_path, output_csv_path)

In [20]:
import pandas as pd

data_df_one = pd.read_csv(output_csv_path, error_bad_lines=False, warn_bad_lines=True)

data_df_one

/var/folders/w_/ccm86j116md1g5bmpdbhyv5r0000gn/T/ipykernel_64045/3324985433.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_df_one = pd.read_csv(output_csv_path, error_bad_lines=False, warn_bad_lines=True)
/var/folders/w_/ccm86j116md1g5bmpdbhyv5r0000gn/T/ipykernel_64045/3324985433.py:3: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_df_one = pd.read_csv(output_csv_path, error_bad_lines=False, warn_bad_lines=True)
Skipping line 244: expected 5 fields, saw 8
Skipping line 245: expected 5 fields, saw 8
Skipping line 246: expected 5 fields, saw 8
Skipping line 247: expected 5 fields, saw 8
Skipping line 248: expected 5 fields, saw 8
Skipping line 249: expected 5 fields, saw 8
Skipping line 250: expected 5 fields, saw 9
Skipping line 251: expected 5 fields, saw 9
Skipping line 252: expected 5

,Undergraduate programmes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Full time,NaN,Repeat Fees 1,NaN
1,NaN,Home/ EU,Overseas,Per Full Unit,Per Half Unit
2,Undergraduate first degree,NaN,NaN,NaN,NaN
3,Students commencing their degree in 2020 (New ...,"£9,250 2","£21,570 3",NaN,NaN
4,Students who commenced their degree between 20...,"£9,250","£20,712","£1,125",£562.50
...,...,...,...,...,...
237,PhD (exceptional registration beyond the fourt...,"£4,407","£19,368","£2,203","£9,684"
238,NaN,NaN,NaN,NaN,NaN
239,NaN,Full time,NaN,Per term,NaN
240,NaN,Home/ EU,Overseas,Home/ EU,Overseas


First get the number of unique courses from our previously cleaned file

In [14]:
csv_path = 'Data/Florian_Wirtz_eigentlich_noch_was_mit_der_v2.csv'
data_df = pd.read_csv(csv_path)

# Combine 'Department' and 'Program' into a new column for unique combinations
data_df['Dept_Program'] = data_df['Department'] + " " + data_df['Program']

# Get unique combinations
unique_dept_programs = data_df['Dept_Program'].unique()

# Output the unique department and programme combinations
unique_dept_programs

array(['Accounting UG Degree', 'Accounting PG Taught',
       'Anthropology UG Degree', 'Economic History UG Degree',
       'Economic History PG Taught', 'Economics UG Degree',
       'Economics PG Taught', 'European Institute PG Taught',
       'Finance UG Degree', 'Finance PG Taught',
       'Gender Studies PG Taught', 'Geography And Environment UG Degree',
       'Geography And Environment PG Taught', 'Government UG Degree',
       'Government PG Taught', 'Health Policy PG Taught',
       'International Development PG Taught',
       'International Relations UG Degree',
       'International Relations PG Taught', 'Law School UG Degree',
       'Law School PG Taught', 'Management UG Degree',
       'Management PG Taught', 'Mathematics UG Degree',
       'Mathematics PG Taught', 'Media And Communications PG Taught',
       'Methodology PG Taught',
       'Philosophy Logic And Scientific Method UG Degree',
       'Philosophy Logic And Scientific Method PG Taught',
       'Psychologica

In [19]:
processed_dept_programs = []
for combo in unique_dept_programs:
    if isinstance(combo, str):  # Check if the item is a string
        if "PG Taught" in combo:
            department = combo.replace(" PG Taught", "")
            new_combo = "MSc in " + department
            processed_dept_programs.append(new_combo)
        elif "UG Degree" in combo:
            # Skip adding to list since we're dropping BSc entries
            continue
            
processed_dept_programs            

['MSc in Accounting',
 'MSc in Economic History',
 'MSc in Economics',
 'MSc in European Institute',
 'MSc in Finance',
 'MSc in Gender Studies',
 'MSc in Geography And Environment',
 'MSc in Government',
 'MSc in Health Policy',
 'MSc in International Development',
 'MSc in International Relations',
 'MSc in Law School',
 'MSc in Management',
 'MSc in Mathematics',
 'MSc in Media And Communications',
 'MSc in Methodology',
 'MSc in Philosophy Logic And Scientific Method',
 'MSc in Psychological And Behavioural Science',
 'MSc in School of Public Policy',
 'MSc in Social Policy',
 'MSc in Sociology',
 'MSc in Statistics',
 'MSc in Anthropology',
 'MSc in International History',
 'MSc in School Of Public Policy',
 'MSc in Media And Communication',
 'MSc in Mathematics  Social Policy',
 'MSc in International..',
 'MSc in Anthropology  Economics',
 'MSc in Accounting  Accounting',
 'MSc in Psychologic..  School Of Pu..',
 'MSc in Management  Social Policy',
 'MSc in Sociology  Statistics'

In [33]:
def find_matching_data(dept_program):
    matches = data_df_one[data_df_one.iloc[:, 0].str.contains(dept_program, na=False)]
    if not matches.empty:
        col2_values = []
        col3_values = []
        for _, row in matches.iterrows():
            # Handle second column, check if value is float and convert to string if not NaN
            val_col2 = row[1]
            try:
                if pd.notna(val_col2):
                    cleaned_value_col2 = int(str(val_col2).replace('£', '').replace(',', ''))
                    col2_values.append(cleaned_value_col2)
            except ValueError:
                col2_values.append(pd.NA)  # Append N/A for non-convertible entries
            
            # Handle third column similarly
            val_col3 = row[2]
            try:
                if pd.notna(val_col3):
                    cleaned_value_col3 = int(str(val_col3).replace('£', '').replace(',', ''))
                    col3_values.append(cleaned_value_col3)
            except ValueError:
                col3_values.append(pd.NA)  # Append N/A for non-convertible entries

        # Calculate averages while ignoring N/A values
        avg_col2 = pd.Series(col2_values).dropna().mean()
        avg_col3 = pd.Series(col3_values).dropna().mean()
        return pd.Series([avg_col2, avg_col3])
    return pd.Series([pd.NA, pd.NA])

# Apply the function to each item in processed_df to create new columns
processed_df[['Avg_Value_2nd_Column', 'Avg_Value_3rd_Column']] = processed_df['Dept_Program'].apply(find_matching_data)

# Display the final DataFrame
processed_df

/var/folders/w_/ccm86j116md1g5bmpdbhyv5r0000gn/T/ipykernel_64045/126614110.py:27: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  avg_col3 = pd.Series(col3_values).dropna().mean()


,Dept_Program,Avg_Value_2nd_Column,Avg_Value_3rd_Column
0,Accounting,28080.0,28464.0
1,Economic History,14640.0,22608.0
2,Economics,23179.0,24134.2
3,European Institute,<NA>,<NA>
4,Finance,28969.5,29185.5
5,Gender Studies,<NA>,<NA>
6,Geography And Environment,<NA>,<NA>
7,Government,NaN,NaN
8,Health Policy,18160.0,25768.0
9,International Development,14640.0,22608.0


In [39]:
import pandas as pd

# Load the CSV file containing the tuition fees and program data
output_csv_path = 'Data/2020_Fees.csv'  # Adjust path as necessary
csv_path = 'Data/Florian_Wirtz_eigentlich_noch_was_mit_der_v2.csv'  # Adjust path as necessary

data_df_one = pd.read_csv(output_csv_path, error_bad_lines=False, warn_bad_lines=True)
data_df = pd.read_csv(csv_path)

# Combine 'Department' and 'Program' into a new column for unique combinations
data_df['Dept_Program'] = data_df['Department'] + " " + data_df['Program']

# Get unique combinations
unique_dept_programs = data_df['Dept_Program'].unique()

# Process only Master's programs and adjust department names
processed_dept_programs = []
for combo in unique_dept_programs:
    if isinstance(combo, str):  # Check if the item is a string
        if "PG Taught" in combo:
            department = combo.replace(" PG Taught", "")
            # Handle specific naming transformations
            if "And" in department:
                department = department.split("And")[0].strip()
            if department == "International History":
                department = "History"
            if department == "European Institute":
                department = "European"
            if department == "Law School" or department == "Law":
                department = "LLM"
            if department == "Philosophy Logic":
                department = "Philosophy"
            if department == "School of Public Policy":
                department = "Public Policy"
            if department == "Gender Studies":
                department = "Gender"
            if "Psychological" in department:
                department = "Psychology"
            processed_dept_programs.append(department)

# Create DataFrame from processed list
processed_df = pd.DataFrame(processed_dept_programs, columns=['Dept_Program'])

def find_matching_data(dept_program):
    matches = data_df_one[data_df_one.iloc[:, 0].str.contains(dept_program, na=False)]
    if not matches.empty:
        col2_values = []
        col3_values = []
        for _, row in matches.iterrows():
            # Process and clean fee data for averaging
            try:
                cleaned_value_col2 = int(str(row[1]).replace('£', '').replace(',', ''))
                col2_values.append(cleaned_value_col2)
            except ValueError:
                col2_values.append(pd.NA)
            try:
                cleaned_value_col3 = int(str(row[2]).replace('£', '').replace(',', ''))
                col3_values.append(cleaned_value_col3)
            except ValueError:
                col3_values.append(pd.NA)
        
        # Calculate averages while ignoring N/A values
        avg_col2 = pd.Series(col2_values).dropna().mean()
        avg_col3 = pd.Series(col3_values).dropna().mean()
        return pd.Series([avg_col2, avg_col3])
    return pd.Series([pd.NA, pd.NA])

# Apply the function to find and average matching tuition fees
processed_df[['Avg_Value_2nd_Column', 'Avg_Value_3rd_Column']] = processed_df['Dept_Program'].apply(find_matching_data)

# Display the final DataFrame
processed_df


/var/folders/w_/ccm86j116md1g5bmpdbhyv5r0000gn/T/ipykernel_64045/1022301878.py:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_df_one = pd.read_csv(output_csv_path, error_bad_lines=False, warn_bad_lines=True)
/var/folders/w_/ccm86j116md1g5bmpdbhyv5r0000gn/T/ipykernel_64045/1022301878.py:7: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_df_one = pd.read_csv(output_csv_path, error_bad_lines=False, warn_bad_lines=True)
Skipping line 244: expected 5 fields, saw 8
Skipping line 245: expected 5 fields, saw 8
Skipping line 246: expected 5 fields, saw 8
Skipping line 247: expected 5 fields, saw 8
Skipping line 248: expected 5 fields, saw 8
Skipping line 249: expected 5 fields, saw 8
Skipping line 250: expected 5 fields, saw 9
Skipping line 251: expected 5 fields, saw 9
Skipping line 252: expected 5

,Dept_Program,Avg_Value_2nd_Column,Avg_Value_3rd_Column
0,Accounting,28080.0,28464.0
1,Economic History,14640.0,22608.0
2,Economics,23179.0,24134.2
3,European,19952.0,22608.0
4,Finance,28969.5,29185.5
5,Gender,14640.0,22608.0
6,Geography,14640.0,22608.0
7,Government,NaN,NaN
8,Health Policy,18160.0,25768.0
9,International Development,14640.0,22608.0
